In [403]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm_notebook as tqdm

In [409]:
NMRC = pd.read_csv('/Users/christian/HOSP10FY2018/hosp10_2018_NMRC.CSV')
NMRC.columns = ['RPT_REC_NUM', 'WKSHT_CD', 'LINE_NUM', 'CLMN_NUM', 'ITM_VAL_NUM']

In [410]:
RPT = pd.read_csv('/Users/christian/HOSP10FY2018/hosp10_2018_RPT.CSV')
RPT.columns = ['RPT_REC_NUM', 'PRVDR_CTRL_TYPE_CD', 'PRVDR_NUM', 'NPI', 'RPT_STUS_CD', 'FY_BGN_DT', 'FY_END_DT', 'PROC_DT', 'INITL_RPT_SW', 'LAST_RPT_SW', 'TRNSMTL_NUM', 'FI_NUM', 'ADR_VNDR_CD', 'FI_CREAT_DT', 'UTIL_CD', 'NPR_DT', 'SPEC_IND', 'FI_RCPT_DT']
RPT = RPT[['RPT_REC_NUM', 'PRVDR_NUM']]

In [411]:
result = pd.merge(NMRC, RPT, how='outer', on='RPT_REC_NUM')

result = result[result['WKSHT_CD'] == 'S300001']

result = result[result['CLMN_NUM'] == '00200']

result = result[result['LINE_NUM'] == 800]

In [412]:
providers = pd.read_csv('/Users/christian/Inpatient_Prospective_Payment_System__IPPS__Provider_Summary_for_All_Diagnosis-Related_Groups__DRG__-_FY2017.csv')

providers = providers[['Provider Id', 'Provider Name', 'Provider Street Address', 'Provider City', 'Provider State', 'Provider Zip Code']].drop_duplicates()

providers.columns = ['PRVDR_NUM', 'PRVDR_NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP']

In [413]:
result_icu = pd.merge(result, providers, how='outer', on='PRVDR_NUM')

result_icu = result_icu.drop_duplicates('PRVDR_NAME')

,RPT_REC_NUM,WKSHT_CD,LINE_NUM,CLMN_NUM,ITM_VAL_NUM,PRVDR_NUM,PRVDR_NAME,ADDRESS,CITY,STATE,ZIP
0,628158.0,S300001,800.0,00200,10.0,250042.0,NORTHWEST MISSISSIPPI MEDICAL CENTER,1970 HOSPITAL DRIVE,CLARKSDALE,MS,38614.0
2,628456.0,S300001,800.0,00200,8.0,140147.0,RICHLAND MEMORIAL HOSPITAL,800 EAST LOCUST,OLNEY,IL,62450.0
4,631016.0,S300001,800.0,00200,12.0,50523.0,SUTTER DELTA MEDICAL CENTER,3901 LONE TREE WAY,ANTIOCH,CA,94509.0
6,631094.0,S300001,800.0,00200,30.0,50305.0,ALTA BATES SUMMIT MEDICAL CENTER - ALTA BATES ...,2450 ASHBY AVENUE,BERKELEY,CA,94705.0
8,631292.0,S300001,800.0,00200,36.0,50043.0,ALTA BATES SUMMIT MEDICAL CENTER,350 HAWTHORNE AVENUE,OAKLAND,CA,94609.0
...,...,...,...,...,...,...,...,...,...,...,...
3561,NaN,NaN,NaN,NaN,NaN,670096.0,"BAY AREA REGIONAL MEDICAL CENTER, LLC",200 BLOSSOM STREET,WEBSTER,TX,77598.0
3562,NaN,NaN,NaN,NaN,NaN,670110.0,FIRST TEXAS HOSPITAL CARRROLLTON LLC,1401 E TRINITY MILLS ROAD,CARROLLTON,TX,75006.0
3563,NaN,NaN,NaN,NaN,NaN,670112.0,CUMBERLAND SURGICAL HOSPITAL,5330 NORTH LOOP 1604 WEST,SAN ANTONIO,TX,78249.0
3564,NaN,NaN,NaN,NaN,NaN,670116.0,WISE HEALTH SYSTEM,3200 NORTH TARRANT PARKWAY,FORT WORTH,TX,76177.0


In [414]:
Hospitals = pd.read_csv('Hospitals.csv')
Hospitals = Hospitals[Hospitals['STATUS'] == 'OPEN']
Hospitals = Hospitals[Hospitals['BEDS'] != '-999']
types = ['GENERAL ACUTE CARE', 'CHILDRENS']
Hospitals = Hospitals[Hospitals.TYPE.isin(types)]
Hospitals = Hospitals.dropna()

In [415]:
result_icu = result_icu[['PRVDR_NUM', 'PRVDR_NAME', 'ITM_VAL_NUM']]
result_icu.columns = ['ProviderNumber', 'NAME', 'ICU_Beds']

In [428]:
result_icu

,ProviderNumber,NAME,ICU_Beds
0,250042.0,NORTHWEST MISSISSIPPI MEDICAL CENTER,10.0
2,140147.0,RICHLAND MEMORIAL HOSPITAL,8.0
4,50523.0,SUTTER DELTA MEDICAL CENTER,12.0
6,50305.0,ALTA BATES SUMMIT MEDICAL CENTER - ALTA BATES ...,30.0
8,50043.0,ALTA BATES SUMMIT MEDICAL CENTER,36.0
...,...,...,...
3561,670096.0,"BAY AREA REGIONAL MEDICAL CENTER, LLC",NaN
3562,670110.0,FIRST TEXAS HOSPITAL CARRROLLTON LLC,NaN
3563,670112.0,CUMBERLAND SURGICAL HOSPITAL,NaN
3564,670116.0,WISE HEALTH SYSTEM,NaN


In [429]:
result_icu.to_csv('ICU_BEDS.csv', index = False)

In [402]:
"""res = pd.merge(Hospitals, result_icu, how = 'left')

res

covid = pd.read_csv('CovidCasesByStateMarch18.csv')

covid = covid[covid['Country/Region'] == 'US']

covid = covid[['Province/State', 'Confirmed', 'Deaths', 'Recovered']]

covid.columns = ['STATE', 'CONFIRMED_CASES', 'DEATHS', 'RECOVERED']

states = pd.read_csv('name-abbr.csv', names = ['STATE', 'ABBREVIATION'])

covid = pd.merge(covid, states)

covid.columns = ['FULL_STATE', 'CONFIRMED_CASES', 'DEATHS', 'RECOVERED', 'STATE']

final = pd.merge(res, covid, how = 'outer', on = 'STATE')

final = final[['LATITUDE', 'LONGITUDE','NAME', 'ADDRESS', 'CITY', 'COUNTY', 'STATE', 'ZIP', 'TELEPHONE',
               'TYPE', 'BEDS', 'ICU_Beds', 'COUNTRY', 'WEBSITE', 'OWNER', 'CONFIRMED_CASES', 'DEATHS', 'RECOVERED']]

final.columns = ['LATITUDE', 'LONGITUDE', 'NAME', 'ADDRESS', 'CITY', 'COUNTY', 'STATE', 'ZIP', 'TELEPHONE', 'TYPE', 
                 'TOTAL_BEDS', 'ICU_BEDS', 'COUNTRY', 'WEBSITE', 'OWNER', 'STATE_CONFIRMED_CASES', 'STATE_DEATHS', 'STATE_RECOVERED']

grouped = final.groupby('STATE')

TotalBedsPerState = grouped['TOTAL_BEDS'].sum().reset_index().to_dict('records')
ICUBedsPerState = grouped['ICU_BEDS'].sum().reset_index().to_dict('records')

final_dicts = final.to_dict('records')
for final_dict in final_dicts:
    for state in TotalBedsPerState:
        if final_dict['STATE'] == state['STATE']:
            final_dict.update({'TOTAL_BEDS_STATE' : state['TOTAL_BEDS']})

for final_dict in final_dicts:
    for state in ICUBedsPerState:
        if final_dict['STATE'] == state['STATE']:
            final_dict.update({'ICU_BEDS_STATE' : state['ICU_BEDS']})

for final_dict in final_dicts:
    try:
        ICU_BedsPerCase = final_dict['ICU_BEDS_STATE'] / final_dict['STATE_CONFIRMED_CASES']
        final_dict.update({'STATE_ICU_BEDS_PER_CONFIRMED_CASE' : ICU_BedsPerCase})
    except:
        final_dict.update({'STATE_ICU_BEDS_PER_CONFIRMED_CASE' : 'NotFound'})

pd.DataFrame(final_dicts).to_csv('covid_march19.csv', index = False)

fin = pd.DataFrame(final_dicts).dropna(thresh = 20)

fin = fin[fin['COUNTRY'] == 'USA']

nans = []
for dicty in fin.to_dict('records'):
    if pd.isna(dicty['ICU_BEDS']):
        nans.append(dicty)

from collections import Counter
Counter(list(pd.DataFrame(nans)['TYPE']))

hospital_ids = pd.read_csv('Hospital_General_Information.csv').iloc[:, 0:5]
hospital_ids.columns = ['ProviderNumber', 'NAME', 'ADDRESS', 'CITY', 'STATE']

hospital_ids['ProviderNumber'] = hospital_ids['ProviderNumber'].astype('str', errors = 'ignore')
result_icu['ProviderNumber'] = result_icu['ProviderNumber'].astype('str', errors = 'ignore')

pd.merge(hospital_ids, result_icu, how = 'inner', on = 'ProviderNumber')

result_icu"""